### Use Birdy to access WPS

In [ ]:
from birdy import WPSClient
url = 'http://localhost:5000/wps'
rooki = WPSClient(url)

### Available processes

In [ ]:
rooki?

### Run subset

In [ ]:
rooki.subset?

In [ ]:
resp_subset = rooki.subset(
    data_ref='cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas',
    time='2085-01-01/2120-12-30',
)

In [ ]:
resp_subset.get()

### metalink output

In [ ]:
resp_subset.get(asobj=True)[1]

### Run average

In [ ]:
rooki.average?

### Run orchestrate workflow

In [ ]:
rooki.orchestrate?

**Workflow with subset and average**

In [ ]:
wf_tree = {
    'doc': "tree workflow",
    'inputs': {
        'tas': ['cmip5.output1.MOHC.HadGEM2-ES.rcp85.mon.atmos.Amon.r1i1p1.latest.tas'],
    },
    "outputs": {
          "output": "average_tas/output"
      },
    'steps': {
        "subset_tas_1": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/tas",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "subset_tas_2": {
              "run": "subset",
              "in": {
                  "data_ref": "subset_tas_1/output",
                  "time": "2090-01-01/2100-12-30"
              }
          },
          "average_tas": {
              "run": "average",
              "in": {
                  "data_ref": "subset_tas_2/output",
                  "axes": "time"
              }
          }
    }}

In [ ]:
import json
wf_tree_json = json.dumps(wf_tree)
wf_tree_json

In [ ]:
resp_wf = rooki.orchestrate(workflow=wf_tree_json)

In [ ]:
resp_wf.get()

**Workflow with diff operator (multiple inputs)**

In [ ]:
wf_tree = {
    'doc': "tree workflow with diff operator",
    'inputs': {
        'inm': ['cmip5.output1.INM.inmcm4.rcp45.mon.ocean.Omon.r1i1p1.latest.zostoga'],
        'mpi': ['cmip5.output1.MPI-M.MPI-ESM-LR.rcp45.mon.ocean.Omon.r1i1p1.latest.zostoga']
    },
    "outputs": {
          "output": "diff/output"
      },
    'steps': {
        "subsetA": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/inm",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "subsetB": {
              "run": "subset",
              "in": {
                  "data_ref": "inputs/mpi",
                  "time": "2085-01-01/2120-12-30"
              }
          },
          "diff": {
              "run": "diff",
              "in": {
                  "data_ref_a": "subsetA/output",
                  "data_ref_b": "subsetB/output",
              }
          }
    }}

In [ ]:
wf_tree_json = json.dumps(wf_tree)
wf_tree_json

In [ ]:
resp_wf = rooki.orchestrate(workflow=wf_tree_json)

In [ ]:
resp_wf.get()